### Add scripts path to the notebook

In [1]:
import sys
import os

current_dir = os.getcwd()
print(current_dir)

# Get the parent directory
parent_dir = os.path.dirname(current_dir)

scripts_path = os.path.join(parent_dir, 'scripts')

# Insert the path to the parent directory
sys.path.insert(0, parent_dir)

# Insert the path to the Scripts directory
sys.path.insert(0, scripts_path)

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join('..')))

d:\KifiyaAIM-Course\Week - 3\ACIS-Car-Insurance-Claim-Analysis\notebooks


### Import Statements

In [2]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scripts.ab_testing import annova_test, test_hypothesis

### Load the data

In [3]:
PATH_TO_DATA = "../data/MachineLearningRating_v3.txt"

In [4]:
data = pd.read_csv(filepath_or_buffer=PATH_TO_DATA, delimiter='|', low_memory=False)

### Test 1

- Null Hypothesis = There are no risk differences across provinces
- Alternate Hypothesis = There are risk differences across provinces

1) Define the KPIs that indicate risk for a given data point 

    From all the columns available TotalClaims and Total Premium are the best to show the risk of someone. I think it is a reasonable metric to tell if someone is a highrisk by seeing their amount of claim, because high claim amount means more acidents and more payout which means a higher risk.      

In [5]:
data[['TotalClaims', 'Province']].isna().mean()

TotalClaims    0.0
Province       0.0
dtype: float64

All of the columns of interest (TotalClaims and Province) don't have null rows so let us proceed to other tasks.

In [6]:
data['Province'].nunique()

9

2) Run the annova test

In [7]:
# calculate the p_value using my custom function that uses annova test
f_statistics, p_value = annova_test(dependent_col='TotalClaims', independent_col='Province', data=data)

3) Accept or reject the hypothesis based on the p_value result

In [9]:
test_hypothesis(null_hypothesis="There are no risk differences across provinces", p_value=p_value)

Rejected the null hypothesis: There are no risk differences across provinces 
p_value: 1.6782057588675903e-07


### Test 2

- Null Hypothesis = There are no risk differences between zip codes
- Alternate Hypothesis = There are risk differences between zip codes

1) Check if there are ZIP codes absent

In [10]:
data['PostalCode'].isna().mean()

np.float64(0.0)

2) Run the Annova test instead of A/B test because there are more than 2 zipcodes/postalcodes

In [13]:
# calculate the p_value using my custom function that uses annova test
f_statistics, p_value = annova_test(dependent_col='TotalClaims', independent_col='PostalCode', data=data)

3) Accept or reject the hypothesis based on the p_value result

In [14]:
test_hypothesis(null_hypothesis="There are no risk differences between zip codes", p_value=p_value)

Accepted the null hypothesis: There are no risk differences between zip codes 
p_value: 0.8906511279164051
